In [11]:
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from zipfile import ZipFile
from xbrl import XBRLParser
from sec_edgar_downloader import Downloader
import pandas as pd
import re
from datetime import datetime

import os
import re
from datetime import datetime
import time

In [12]:
def download_filings_for_specific_years(ticker, filing_type, years):
    dl = Downloader()

    # Get current year
    current_year = datetime.now().year

    for year in years:
        if year <= current_year:
            # Convert datetime to string
            before_date = datetime(year+1, 1, 1).strftime('%Y-%m-%d')
            after_date = datetime(year, 1, 1).strftime('%Y-%m-%d')

            dl.get(filing_type, ticker, amount=1, after=after_date, before=before_date)
            time.sleep(1)  # Add a delay to prevent hitting server too frequently




In [13]:
# Specify the years for which you want to download filings
years = [2020, 2021, 2022, 2023]

# Call the function for each filing type
download_filings_for_specific_years("AAPL", "10-K", years)
download_filings_for_specific_years("AAPL", "10-Q", years)


In [16]:
import os
import pandas as pd
from bs4 import BeautifulSoup

# Function to extract tables from an html file
def extract_financial_tables(filepath, table_names):
    with open(filepath, 'r') as f:
        soup = BeautifulSoup(f, 'html.parser')
    
    # Find tables
    tables = soup.find_all('table')

    # Data structure to hold our tables
    extracted_tables = {}

    # Loop over tables and check if title matches table names
    for table in tables:
        for name in table_names:
            if name.lower() in table.get_text(separator=' ', strip=True).lower():
                # Convert the table to a DataFrame
                df = pd.read_html(str(table), flavor='bs4')[0]

                # Add the DataFrame to our data structure
                extracted_tables[name] = df
                break

    return extracted_tables

# Define your path and table names
filepath = 'sec-edgar-filings/AAPL/10-K/0000320193-20-000096/full-submission.txt'
table_names = ['consolidated balance sheets', 'consolidated statements of income', 'consolidated statements of cash flows']

# Call the function
tables = extract_financial_tables(filepath, table_names)

# Print the extracted tables
for name, table in tables.items():
    print(f"{name}:\n{table}\n\n")


consolidated balance sheets:
                                                   0  \
0                                                NaN   
1         Index to Consolidated Financial Statements   
2  Consolidated Statements of Operations for the ...   
3  Consolidated Statements of Comprehensive Incom...   
4  Consolidated Balance Sheets as of September 26...   
5  Consolidated Statements of Shareholders’ Equit...   
6  Consolidated Statements of Cash Flows for the ...   
7         Notes to Consolidated Financial Statements   
8  Selected Quarterly Financial Information (Unau...   
9  Reports of Independent Registered Public Accou...   

                                                   1  \
0                                                NaN   
1         Index to Consolidated Financial Statements   
2  Consolidated Statements of Operations for the ...   
3  Consolidated Statements of Comprehensive Incom...   
4  Consolidated Balance Sheets as of September 26...   
5  Consolidated St

In [35]:
import pandas as pd
from bs4 import BeautifulSoup


def extract_financial_statements(filepath):
    with open(filepath, 'r') as f:
        soup = BeautifulSoup(f, 'html.parser')
    
    # Find the XBRL document URL
    xbrl_link = soup.find('a', {'title': 'XBRL Document'})['href']
    xbrl_url = f"https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/aapl-20200926.htm#ief781ab58e4f4fcaa872ddbd30da40e1_85"
    
    # Download the XBRL document
    xbrl_content = pd.read_html(xbrl_url)
    
    # Extract the financial statements
    statements = {
        'Income Statement': None,
        'Balance Sheet': None,
        'CONSOLIDATED STATEMENTS OF CASH FLOWS - USD ($)': None
    }
    
    for table in xbrl_content:
        if 'income' in table.values[0, 0].lower():
            statements['Income Statement'] = table
        elif 'balance sheet' in table.values[0, 0].lower():
            statements['Balance Sheet'] = table
        elif 'cash flow' in table.values[0, 0].lower():
            statements['Cash Flow Statement'] = table
    
    return statements


# Specify the file path of the 10-K filing
filepath = '/Users/charlesacevedo/HiFiAi/sec-edgar-filings/AAPL/10-K/0000320193-20-000096/full-submission.txt'

# Extract the financial statements
statements = extract_financial_statements(filepath)

# Print the extracted financial statements
for name, table in statements.items():
    print(f"{name}:")
    if table is None:
        print("Statement not found.")
    else:
        print(table)
    print("\n")



TypeError: 'NoneType' object is not subscriptable

In [30]:
financial_statements

{}

In [28]:
def convert_to_dataframe(text):
    lines = text.split('\n')
    
    # Define list to store parsed lines
    parsed_lines = []
    
    # Iterate over each line
    for line in lines:
        # Use regex to split line into account name and amounts
        split_line = re.split(r'\s*(\$\d{1,3}(,\d{3})*(\.\d{1,2})?)', line)
        
        # Remove any empty strings from list
        split_line = list(filter(None, split_line))
        
        # If regex successfully split the line into three parts, append to parsed_lines
        if len(split_line) == 3:
            parsed_lines.append(split_line)
    
    # Convert parsed_lines into DataFrame and return
    return pd.DataFrame(parsed_lines, columns=['Account', '2022', '2021'])


df = convert_to_dataframe(balance_sheet_data)


NameError: name 'balance_sheet_data' is not defined

In [20]:
print(df)

Empty DataFrame
Columns: [Account, 2022, 2021]
Index: []


In [21]:
balance_sheet_data

'(In millions, except number of shares which are reflected in thousands and par value)September 24,2022September 25,2021ASSETS:Current assets:Cash and cash equivalents$23,646\xa0$34,940\xa0Marketable securities24,658\xa027,699\xa0Accounts receivable, net28,184\xa026,278\xa0Inventories4,946\xa06,580\xa0Vendor non-trade receivables32,748\xa025,228\xa0Other current assets21,223\xa014,111\xa0Total current assets135,405\xa0134,836\xa0Non-current assets:Marketable securities120,805\xa0127,877\xa0Property, plant and equipment, net42,117\xa039,440\xa0Other non-current assets54,428\xa048,849\xa0Total non-current assets217,350\xa0216,166\xa0Total assets$352,755\xa0$351,002\xa0LIABILITIES AND SHAREHOLDERS’ EQUITY:Current liabilities:Accounts payable$64,115\xa0$54,763\xa0Other current liabilities60,845\xa047,493\xa0Deferred revenue7,912\xa07,612\xa0Commercial paper9,982\xa06,000\xa0Term debt11,128\xa09,613\xa0Total current liabilities153,982\xa0125,481\xa0Non-current liabilities:Term debt98,959\xa